# ---Day 11: Space Police---

## ---Part One---

On the way to Jupiter, you're <a src="https://www.youtube.com/watch?v=KwY28rpyKDE">pulled over</a> by the ***Space Police***.

"Attention, unmarked spacecraft! You are in violation of Space Law! All spacecraft must have a clearly visible ***registration identifier***! You have 24 hours to comply or be sent to <a src="https://www.youtube.com/watch?v=BVn1oQL9sWg&t=5">Space Jail!</a>

Not wanting to be sent to Space Jail, you radio back to the Elves on Earth for help. Although it takes almost three hours for their reply signal to reach you, they send instructions for how to power up the ***emergency hull painting robot*** and even provide a small Intcode program (day09) (your puzzle input) that will cause it to paint your ship appropriately.

There's just one problem: you don't have an emergency hull painting robot.

You'll need to build a new emergency hull painting robot. The robot needs to be able to:
1. move around on the grid of square panels on the side of your ship,
2. detect the color of its current panel, and
3. paint its current panel ***black*** or ***white***. (All of the panels are currently ***black***.)

The Intcode program will serve as the brain of the robot. The program uses input instructions to access the robot's camera: provide `0` if the robot is over a ***black*** panel or `1` if the robot is over a ***white*** panel. Then, the program will output two values:

* First, it will output a value indicating the ***color to paint the panel*** the robot is over: `0` means to paint the panel ***black***, and `1` means to paint the panel ***white***.
* Second, it will output a value indicating the ***direction the robot should turn***: `0` means it should turn ***left 90 degrees***, and `1` means it should turn ***right 90 degrees***.
After the robot turns, it should always move ***forward exactly one panel***. The robot starts facing ***up***.

The robot will continue running for a while like this and halt when it is finished drawing. Do not restart the Intcode computer inside the robot during this process.

For example, suppose the robot is about to start running. Drawing black panels as `.`, white panels as `#`, and the robot pointing the direction it is facing (`< ^ > v`), the initial state and region near the robot looks like this:
```
.....
.....
..^..
.....
.....
```
The panel under the robot (not visible here because a `^` is shown instead) is also black, and so any input instructions at this point should be provided `0`. Suppose the robot eventually outputs `1` (paint white) and then `0` (turn left). After taking these actions and moving forward one panel, the region now looks like this:
```
.....
.....
.<#..
.....
.....
```
Input instructions should still be provided `0`. Next, the robot might output `0` (paint black) and then `0` (turn left):
```
.....
.....
..#..
.v...
.....
```
After more outputs (`1,0`, `1,0`):
```
.....
.....
..^..
.##..
.....
```
The robot is now back where it started, but because it is now on a white panel, input instructions should be provided `1`. After several more outputs (`0,1`, `1,0`, `1,0`), the area looks like this:
```
.....
..<#.
...#.
.##..
.....
```
Before you deploy the robot, you should probably have an estimate of the area it will cover: specifically, you need to know ***the number of panels it paints at least once***, regardless of color. In the example above, the robot painted ***`6` panels*** at least once. (It painted its starting panel twice, but that panel is <a src="https://www.youtube.com/watch?v=KjsSvjA5TuE"> still only counted once</a>; it also never painted the panel it ended on.)

Build a new emergency hull painting robot and run the Intcode program on it. **How many panels does it paint at least once?**

### Day09 Computer

In [1]:
class Computer:
    def __init__(self, program, input_val=1):
        self.program = program + [0 for i in range(1000)]
        self.input = input_val
        self.pointer = 0
        self.instruction = 0
        self.base = 0
        self.out = 0
        self.opcode = { 1 : self.add,
                       2 : self.multiply,
                       3 : self.save,
                       4 : self.output,
                       5 : self.jump_true,
                       6 : self.jump_false,
                       7 : self.less_than,
                       8 : self.is_equal,
                       9 : self.change_base,
                       99 : self.halt }
    
    def set_instruction(self):
        self.instruction = self.program[self.pointer]
        self.pointer += 1
        
    def get_opcode(self):
        return self.instruction % 100
    
    def get_modes(self):
        return self.instruction // 100

    def get_parameters(self, modes, num_parameters=3):
        mode = [ (modes // 10**n % 10) for n in range(num_parameters) ]
        parameters = []
        for i in range(self.pointer, self.pointer+num_parameters):
            if mode[i-self.pointer]:
                if mode[i-self.pointer] == 2:
                    parameters.append(self.program[self.program[i]+self.base])
                else:
                    parameters.append(self.program[i])
            else:
                parameters.append(self.program[self.program[i]])
        
        return parameters
    
    def set_write(self, modes, num_parameters):
        relative = [ (modes // 10**n % 10) for n in range(num_parameters) ][-1]
        if relative:
            return self.program[self.pointer+num_parameters-1]+self.base
        return self.program[self.pointer+num_parameters-1]

    def add(self, step=3):
        values = self.get_parameters(self.get_modes(), step)
        # Write over last parameter because it should always be a value indicating the location
        values[-1] = self.set_write(self.get_modes(), step)
        self.program[values[-1]] = values[0] + values[1]
        self.pointer += step
        
    def multiply(self, step=3):
        values = self.get_parameters(self.get_modes(), step)
        # Write over last parameter because it should always be a value indicating the location
        values[-1] = self.set_write(self.get_modes(), step)
        self.program[values[-1]] = values[0] * values[1]
        self.pointer += step
        
    def save(self, step=1):
        if self.get_modes():
            self.program[self.program[self.pointer]+self.base] = self.input
        else:
            self.program[self.program[self.pointer]] = self.input
        self.pointer += step
        
    def output(self, step=1):
        self.out = self.get_parameters(self.get_modes(), step)[0]
        print(f'Output: {self.out}')
        self.pointer += step 
        
    def jump_true(self, step=2):
        values = self.get_parameters(self.get_modes(), step)
        if values[0] != 0:
            self.pointer = values[1]
        else:
            self.pointer += step
            
    def jump_false(self, step=2):
        values = self.get_parameters(self.get_modes(), step)
        if values[0] == 0:
            self.pointer = values[1]
        else:
            self.pointer += step
            
    def less_than(self, step=3):
        values = self.get_parameters(self.get_modes(), step)
        # Write over last parameter because it should always be a value indicating the location
        values[-1] = self.set_write(self.get_modes(), step)
        self.program[values[-1]] = int(values[0] < values[1])
        self.pointer += step
    
    def is_equal(self, step=3):
        values = self.get_parameters(self.get_modes(), step)
        # Write over last parameter because it should always be a value indicating the location
        values[-1] = self.set_write(self.get_modes(), step)
        self.program[values[-1]] = int(values[0]==values[1])
        self.pointer += step
    
    def change_base(self, step=1):
        value = self.get_parameters(self.get_modes(), step)
        self.base += value[0]
        self.pointer += step
        
    def halt(self):
        print("Program Halted.")
        self.instruction = -1
    
    def run(self):
        while self.instruction != -1:
            #print(self.program, self.base) # Used for testing
            self.set_instruction()
            #print(self.pointer, self.instruction) # Used for Testing
            self.opcode[self.get_opcode()]()
            # print(self.program[:self.pointer+1], self.program[self.pointer:self.pointer+4]) # Used for Testing

In [2]:
class Painter(Computer):
    def __init__(self, program, start_tile=0):
        Computer.__init__(self, program, input_val=start_tile)
        self.direction = 0 # 0,1,2,3 = N,E,S,W
        self.x = 0
        self.y = 0
        self.paint = True
        self.grid = {(0,0): self.input}
        self.move = { 0:(0,1), \
                     1:(1,0), \
                     2:(0,-1), \
                     3:(-1,0 ) }
    
    def get_color(self, step=1):
        #used to get color of tile
        try:
            return self.grid[(self.x,self.y)]
        except:
            return 0
        
    def save(self, step=1):
        #overrite Computer save to save the tile color
        if self.get_modes():
            self.program[self.program[self.pointer]+self.base] = self.get_color()
        else:
            self.program[self.program[self.pointer]] = self.get_color()
        self.pointer += step
        
    def turn_and_move(self, right):
        if right:
            self.direction = (self.direction + 1) % 4
        else:
            self.direction = (self.direction - 1) % 4
        self.x = self.x + self.move[self.direction][0]
        self.y = self.y + self.move[self.direction][1]
        
    def output(self, step=1):
        if self.paint:
            self.grid[(self.x,self.y)] = self.get_parameters(self.get_modes(), step)[0]
            #print(self.grid)
        else:
            self.turn_and_move(self.get_parameters(self.get_modes(), step)[0])
        self.paint = not self.paint
        self.pointer += step
    
    def run(self):
        while self.instruction != -1:
            #print(self.pointer)
            #print(self.pointer, self.program[:self.pointer+1])
            self.set_instruction()
            self.opcode[self.get_opcode()]()

In [3]:
with open('day11.txt') as f:
    program = [ int(i) for i in f.read().strip().split(',') ]
    bot = Painter(program)
    bot.run()
    print(f'The bot painted {len(bot.grid)} panels')

Program Halted.
The bot painted 2428 panels


#### Test Case

In [4]:
test = [3,100,104,1,104,0,3,100,104,0,104,0,3,100,104,1,104,0,3,100,104,1,104,0 \
        ,3,100,104,0,104,1,3,100,104,1,104,0,3,100,104,1,104,0,99]
test_bot = Painter(test)
test_bot.run()
test_bot.x, test_bot.y, test_bot.grid

Program Halted.


(0, 1, {(0, 0): 0, (-1, 0): 0, (-1, -1): 1, (0, -1): 1, (1, 0): 1, (1, 1): 1})

## ---Part Two---

You're not sure what it's trying to paint, but it's definitely not a ***registration identifier***. The Space Police are getting impatient.

Checking your external ship cameras again, you notice a white panel marked "emergency hull painting robot starting panel". The rest of the panels are ***still black***, but it looks like the robot was expecting to ***start on a white panel***, not a black one.

Based on the Space Law Space Brochure that the Space Police attached to one of your windows, a valid registration identifier is always ***eight capital letters***. After starting the robot on a single white panel instead, **what registration identifier does it paint on your hull?**

In [5]:
with open('day11.txt') as f:
    program = [ int(i) for i in f.read().strip().split(',') ]
    bot = Painter(program, start_tile=1)
    bot.run()

Program Halted.


In [6]:
#get grid dimensions, initialize blank grid
print("X Range:", min(bot.grid, key=lambda x: x[0])[0], max(bot.grid, key=lambda x: x[0])[0])
print("Y Range:", min(bot.grid, key=lambda y: y[1])[1], max(bot.grid, key=lambda y: y[1])[1])

X Range: 0 42
Y Range: -5 0


In [7]:
#initialize blank grid
blank = [ [ '.' for x in range(0,43)] for y in range(-5,1) ]

In [8]:
for panel in bot.grid:
    if bot.grid[panel]:
        blank[panel[1]+5][panel[0]] = '#'

code = [ ''.join(line) for line in blank ]
for i in range(len(code)): print(code.pop()) 

.###....##.#....####.###..#..#..##..#..#...
.#..#....#.#....#....#..#.#..#.#..#.#..#...
.#..#....#.#....###..###..#..#.#....#..#...
.###.....#.#....#....#..#.#..#.#....#..#...
.#.#..#..#.#....#....#..#.#..#.#..#.#..#...
.#..#..##..####.#....###...##...##...##....
